# Chains

# 準備

In [64]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["API_KEY"]

# LLM Chain：Chainの基本的な使い方

In [63]:
from langchain import PromptTemplate, OpenAI, LLMChain

template = "{subject}を勉強する方法をステップ・バイ・ステップで教えてください"

llm = OpenAI(max_tokens=300, temperature=1.2)

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template),
)

llm_chain.run("英語")

'。\n\n１．まず、一般的な単語や熟語を選び、暗記していきましょう。\n\n２．暗記で増やした単語を使って、小さい文をつくりましょう。\n\n３．文章を読む練習をしましょう。英語で書かれた文章などを選んで読んで、物語を映像的にイメージしたりしましょう。\n\n４．会話の練習をしましょう。他の話者とのディスカッションなど英語で会話できるようになるのに役立ちます。オンラインの会話パートナーを探すサイトもあります。\n\n５．'

# Conversation Chain：会話を続ける方法

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

model_name = "gpt-3.5-turbo"

llm = ChatOpenAI(
    model_name=model_name,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=1.2,
)

conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())

In [7]:
while True:
    # ユーザーからの質問を受付
    message = input("メッセージを入力:")

    # 質問が入力されなければ終了
    if message.strip() == "":
        break
    display(f"質問:{message}")

    # 言語モデルに質問
    conversation.predict(input=message)

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは！'

こんにちは！元気ですか？お話は何ですか？

'質問:仙台でおすすめのランチを教えて！'

仙台でおすすめのランチはたくさんありますよ！例えば、仙台駅近くには美味しいラーメン店がたくさんあります。"牛タンラーメン"や"味噌ラーメン"は特におすすめです。また、仙台の地元特産品である"ずんだ"を使ったスイーツや料理も試してみる価値があります。太白区の"青葉通り一番町商店街"や宮城野区の"イオンモール仙台泉店"などにも美味しいランチを提供しているレストランやカフェがありますので、チェックしてみてください！

'質問:ずんだって何ですか？辛いものですか？'

ずんだは、宮城県仙台市やその周辺で特産される地元の食材です。実は、ずんだは甘味です。主に枝豆を練りつぶして作られる緑色のペーストで、抹茶風味の甘さが特徴です。一般的にデザートや和菓子に使用されます。ずんだの甘さは辛さではなく、苦味ともまた違いますので、辛いものではありませんよ。周辺の飲食店や土産物店で買えるし、仙台のご当地グルメとしても人気があります！
---ご利用ありがとうございました！---


# RetrievalQA Chain：Index＋言語モデル

In [8]:
from langchain.document_loaders.directory import DirectoryLoader

# テキストファイルを読込
loader = DirectoryLoader("./data/")
documents = loader.load()

In [16]:
from langchain.text_splitter import CharacterTextSplitter

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [17]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

In [31]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

# RetrievalQAチェーンの作成
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=1.2), chain_type="stuff", retriever=db.as_retriever()
)

In [32]:
query = "キノコストレージについて説明して"
qa.run(query)

' キノコストレージとは、容量の大きなデータストレージ装置を指す架空のIT用語です。この装置は非常に小さなサイズながら、膨大なデータを保持することができるとされています。また、キノコストレージは高い耐久性を持ち、データの保護と安全性を確保すると説明されます。'

# 対話型 RetrievalQA Chain

In [65]:
from langchain.document_loaders import WikipediaLoader
from langchain.document_loaders import PyPDFLoader

# 複数のローダーから読込
loaders = [
    WikipediaLoader(query="SDGs", load_max_docs=3, lang="ja"),
    PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")]

documents = []

for loader in loaders:
    documents.extend(loader.load())

In [76]:
documents

[Document(page_content='持続可能な開発目標（じぞくかのうなかいはつもくひょう、英語:  Sustainable Development Goals、略称: SDGs（エスディージーズ））は、2015年9月25日に国連総会で採択された、持続可能な開発のための17の国際目標である。その下に、169の達成基準と232の指標が決められている。\n\nミレニアム開発目標 (MDGs: Millennium Development Goals) が2015年に終了することに伴い、2015年9月25日の国連総会で、『我々の世界を変革する：持続可能な開発のための2030アジェンダ』（Transforming our world: the 2030 Agenda for Sustainable Development、または単に2030 Agendaとも）が採択された。これは、持続可能な開発のために必要不可欠な、向こう15年間の新たな行動計画である。その中で、2030年までに達成するべき持続可能な開発目標 (SDGs) として17の世界的目標と169の達成基準が示された。\n\n\n== 17の目標 ==\n\nSDGsは、以下の17の目標から構成されている。その他、169項目など全文はウィキソースの「持続可能な開発のための2030アジェンダ」を参照。\n\n\n== 進捗状況 ==\n2017年7月 - 国際連合の事務総長であるアントニオ・グテーレスは、SDGsに掲げられている多くの分野の前進が2030年までに達成できるペースをはるかに下回っているとし、前進を加速すべく取り組みを強化する必要があるとする国連報告書を発表した。\n2019年5月 - 国際連合事務総長のアントニオ・グテーレスは、経済社会理事会ECOSOCに提出する「国連持続可能な開発目標SDGs進捗報告書」を公表する。首脳レベルのSDGs進捗状況は4年に1度の発表であるため、2015年に採択されてから初の報告として注目を集める。目標1から17まで各々の課題が発表されたが、全体的に目標達成に至らず、掲げる目標の達成までに課題が山積みしている。\n2020年9月 - 国際連合副事務総長のアミナ・モハメドは、新型コロナウイルス感染症の世界的流行が原因で、貧困撲滅を目指す目標1の取り組みが大幅に減退して

In [66]:
from langchain.text_splitter import CharacterTextSplitter

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [67]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Indexの構築
db = Chroma.from_documents(texts, OpenAIEmbeddings())

In [70]:
from langchain.memory import ConversationBufferMemory

# Memoryの作成
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [71]:
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# 会話型RetrievalQAチェーンの作成
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=1.2),
    retriever=db.as_retriever(),
    max_tokens_limit=4096,
    memory=memory)

In [72]:
# ユーザーからの質問
query = "SDGsとは何ですか？"
result = qa({"question": query})

# 言語モデルからの回答を表示
result["answer"]

' SDGsとは、持続可能な開発のための17の国際目標であり、2015年9月25日に国連総会で採択されたものです。その下に、169の達成基準と232の指標が決められています。'

In [73]:
# ユーザーからの質問
query = "1つ目の目標について説明してください。出典も示してください"
result = qa({"question": query})

# 言語モデルからの回答を表示
result["answer"]

' SDGsで最初の目標は「貧困を終わらせる」です。詳しい説明は、SDGsに関連するウィキソースの『Transforming our world: 2030 Agenda for Sustainable Development』から抜粋した情報です: "SDGsの目標1: 貧しい人たちを支援して、貧困を実質的に終わらせ、全人口の向上と暮らしやすい社会を支援するために、貧困の根本原因を特に注力し、手助けする。しかしながら、高齢者を含む様々な世代のあらゆる人々にとって、'

In [74]:
# ユーザーからの質問
query = "オープンデータの定義とは？"
result = qa({"question": query})

# 言語モデルからの回答を表示
result["answer"]

' オープンデータとは、国、地方公共団体及び事業者が保有する官民データのうち、国民誰もがインターネット等を通じて容易に利用（加工、編集、再配布等）できるよう、①営利目的、非営利目的を問わず二次利用可能なルールが適用されたもの、②機械判読に適したもので定義されるデータのことです。'

In [75]:
# 保存されているメッセージを取得
buffer = memory.load_memory_variables({})
print(buffer)

{'chat_history': [HumanMessage(content='SDGsとは何ですか？', additional_kwargs={}, example=False), AIMessage(content=' SDGsとは、持続可能な開発のための17の国際目標であり、2015年9月25日に国連総会で採択されたものです。その下に、169の達成基準と232の指標が決められています。', additional_kwargs={}, example=False), HumanMessage(content='1つ目の目標について説明してください。出典も示してください', additional_kwargs={}, example=False), AIMessage(content=' SDGsで最初の目標は「貧困を終わらせる」です。詳しい説明は、SDGsに関連するウィキソースの『Transforming our world: 2030 Agenda for Sustainable Development』から抜粋した情報です: "SDGsの目標1: 貧しい人たちを支援して、貧困を実質的に終わらせ、全人口の向上と暮らしやすい社会を支援するために、貧困の根本原因を特に注力し、手助けする。しかしながら、高齢者を含む様々な世代のあらゆる人々にとって、', additional_kwargs={}, example=False), HumanMessage(content='オープンデータの定義とは？', additional_kwargs={}, example=False), AIMessage(content=' オープンデータとは、国、地方公共団体及び事業者が保有する官民データのうち、国民誰もがインターネット等を通じて容易に利用（加工、編集、再配布等）できるよう、①営利目的、非営利目的を問わず二次利用可能なルールが適用されたもの、②機械判読に適したもので定義されるデータのことです。', additional_kwargs={}, example=False)]}


# Summarize Chain：文章の要約

In [77]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# PDFの読込
loader = PyPDFLoader("./data2/20210615_resources_data_guideline_01.pdf")
texts = loader.load()

# チャンクに分割
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [38]:
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)

chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(texts)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 22549 tokens (22293 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [39]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)

' 1994年に、文化的景観という概念が導入され、自然資源の持続可能な利用を重視した世界遺産リストが紹介された。この概念は日本の文化遺産保護に影響を与え、パラチとウーロプレトの会議で持続可能な開発の方向性が確認された。日本では、政府と民間データ利用促進基本法、「5つ星」評価システム、オープンデータガイド、オープンデータ・バイ・デザイン2などのルール'

# Sequential Chain

### SimpleSequentialChain

In [53]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMCheckerChain, SimpleSequentialChain

llm = OpenAI(temperature=0.7)

# 1つ目のChain
template = "{subject}に関する問題を作成してください。答えは作成しないでください"
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template),
)

# 2つ目のChain
check_chain = LLMCheckerChain.from_llm(
    llm=llm)

# Chain
overall_chain = SimpleSequentialChain(
    chains=[llm_chain, check_chain],
    verbose=True)

In [54]:
# Simple Sequential Chainの実行
overall_chain.run("動物")



> Entering new  chain...
。

Q. イヌの種類として有名なのは何か？


> Entering new  chain...

> Finished chain.
 イヌの種類として有名なものとして、シェパード犬、チワワ、ラブラドール・レトリバー、ポメラニアン、ゴールデン・レトリバーなどがあります。

> Finished chain.


' イヌの種類として有名なものとして、シェパード犬、チワワ、ラブラドール・レトリバー、ポメラニアン、ゴールデン・レトリバーなどがあります。'

### SequentialChain

In [78]:
from langchain import PromptTemplate

# 1つ目のChain
template1 = "{theme}に関する{target}向けのブログ記事のタイトルを考えてください。"
llm_chain1 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template1),
    output_key="title"
)

# 2つ目のChain
template2 = "ブログ記事「{title}」の見出しを考えてください。"
llm_chain2 = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(template2),
    output_key="headline"
)

# Chain
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[llm_chain1, llm_chain2],
    input_variables=["theme", "target"],
    output_variables=["title", "headline"],
    verbose=True)

In [80]:
# Chainの実行
overall_chain({
    "theme":"観光",
    "target": "日本の富裕層"})



> Entering new  chain...

> Finished chain.


{'theme': '観光',
 'target': '日本の富裕層',
 'title': '\n\n1. 世界最高峰の贅沢旅行：ギネスに認定された史上最高のモスクワ観光\n2. 絶対に衝撃を受ける素晴らしいツアー：ロンドンで名所を巡る特別なツアー\n3. 不思議な文化を体験する：バンクーバーで聞ける美しい伝統的な音楽\n4. 真のハイルームを実現する独占的なビジネストラベル：パリと東京を繋ぐ専用のプライベートジェット\n5. 完璧なバケーション体験：南中国の水上ビ忘れられないアクティビティ',
 'headline': '\n\n1.究極の贅沢旅行：ギネス記録のモスクワ観光へようこそ\n2. 衝撃的な一歩：ロンドンの伝統的な名所ツアーを是非\n3. 生き生きとした文化に没頭：バンクーバーにある美しい伝統音楽をお楽しみください\n4. 異国の究極のビジネス旅行：パリと東京をお楽しみください\n5. 究極のバケーション：中国南方の水上で不忘なアクティビティをもう一度'}